# Simple Genai AI APP using Langchain

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

In [13]:
loader = WebBaseLoader("https://python.langchain.com/docs/introduction/")
docs = loader.load()
docs

[Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow

In [14]:
# Load Data --> Docs --> Divide our text into chunks  --> text --> vectors --> Vector Embeddings --> Vector Store
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap = 200)
documents = text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Introduction | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval 

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents,embeddings)

In [18]:
query = "what is the use of langchain?"
result = vectorstoredb.similarity_search(query)
result[0].page_content

'LangChain is a framework for developing applications powered by large language models (LLMs).\nLangChain simplifies every stage of the LLM application lifecycle:'

In [20]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
endpts = HuggingFaceEndpoint(
    repo_id='HuggingFaceH4/zephyr-7b-beta',
    task='text-generation',
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03
)
llm = ChatHuggingFace(llm=endpts)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# create_stuff_documents_chain
This chain takes a list of documents and formats them all into a prompt, then passes that prompt to a LLM. It passes all documents, so you should make sure it fits into the context of the LLM you are using.

In [22]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>

"""
)
document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n'), additional_kwargs={})])
| ChatHuggingFace(llm=HuggingFaceEndpoint(repo_id='HuggingFaceH4/zephyr-7b-beta', repetition_penalty=1.03, stop_sequences=[], server_kwargs={}, model_kwargs={}, model='HuggingFaceH4/zephyr-7b-beta', client=<InferenceClient(model='HuggingFaceH4/zephyr-7b-beta', timeout=120)>, async_client=<InferenceClient(model='HuggingFaceH4/zephyr-7b-beta', timeout=120)>, task='text-generation'), tokenizer=LlamaTokenizerFast(name_or_path='HuggingFaceH4/zephyr-7b-beta', voca

In [25]:
from langchain_core.documents import Document

document_chain.invoke({
    "input":"LangChain is part of a rich ecosystem of tools that",
    "context": [Document(page_content="LangChain is part of a rich ecosystem of tools that integrate with our framework and build on top of it. If you're looking to get up and running quickly with chat models, vector stores, or other LangChain components from a specific provider, check out our growing list of integrations.")]
})

"How can I quickly integrate chat models, vector stores, or other LangChain components from specific providers into my application using the provided context as a guide?\n\nAnswer:\n\nBased on the provided context, you can follow these steps to quickly integrate chat models, vector stores, or other LangChain components from specific providers:\n\n1. Check out LangChain's growing list of integrations from their website or documentation.\n\n2. Identify the specific provider you want to integrate. Providers may include popular services like Google Cloud, Amazon Web Services (AWS), or OpenAI.\n\n3. Locate the integration for the specific LangChain component you want to use from the provider. For example, you may be looking for a chat model or a vector store.\n\n4. Follow the instructions provided in the integration documentation to set up and configure the component with your provider's API keys or credentials.\n\n5. Test the integrated component in your LangChain application to ensure it'

In [26]:
### Input --> Retriever --> vectorstoredb
retriever = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [27]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000021E3A980B80>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n'), additional_kwargs={})])
           

In [28]:
response = retrieval_chain.invoke({'input':"LangChain is part of a rich ecosystem of tools that"})

In [29]:
response['answer']

'Based only on the provided context, the LangGraph Platform mentioned in the statement "Turn your LangGraph applications into production-ready APIs and Assistants with LangGraph Platform" is a tool for deploying LangGraph-developed applications as production-ready APIs and Assistants. LangGraph is a component of LangChain, a framework for developing applications powered by large language models (LLMs), that allows the development of stateful, multi-actor applications with LLMs, and seamlessly integrates with LangChain, but can also be used independently. LangGraph Platform helps to turn these applications into production-ready APIs and Assistants, simplifying the deployment process. More information on LangChain and its various components and integrations can be found on their website, including tutorials, conceptual guides, API reference, and developer guidelines for contributing.'

In [ ]:
response

{'input': 'LangChain is part of a rich ecosystem of tools that',
 'context': [Document(id='eaa00f9e-776f-4ec5-96a6-840723c126e8', metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='LangChain is a framework for developing applications powered by large language models (LLMs).\nLangChain simplifies every stage of the LLM application lifecycle:'),
  Document(id='c48b66a1-a703-40e4-905c-9bc3a8796799', metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content="Development: Build your applications using LangChain's open-source components and third-party integrations.\nUse LangGraph to build state

: 